In [1]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

# 1. Multiplying Matrices

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(key)
print(x)

[0 0]
[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


Time taken for multiply matrices on JAX and Numpy

In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

2.65 ms ± 36 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)

# Note here we actually using jnp to compute np array. In this case, np array should first been move to CUDA
%timeit jnp.dot(x, x.T).block_until_ready()

20.4 ms ± 475 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Use `device_put` to ensure NDArray is backed by the device memory

In [5]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

2.62 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 2. Using `jit()` to speed up function

- JAX dispatches kernels to GPU one operation at a time;
- `@jit` helps compile multiple operations together using XLA

In [6]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

1.39 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

523 µs ± 292 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# 3. Taking derivatives with `grad()`

In [8]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


In [16]:
# to verify the gradient
def first_finite_differences(f, x):
    eps = 1e-3
    return jnp.array([
        (f(x + eps * v) - f(x - eps * v)) / (2 * eps) for v in jnp.eye(len(x))
    ])

print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


# 4. Auto-vectorization with `vmap()`

In [24]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return jnp.dot(mat, v)

In [25]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)

%timeit batched_apply_matrix(batched_x).block_until_ready()

889 µs ± 32.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

870 µs ± 29.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
